In [1]:
import sys
sys.path.append("fwks_lib")

# limit the memory usage
import keras.backend as K
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from fwks.tasks import FeatureLearnabilityTask, Task
from dissertation_model import (
    mfcc_model,
    handcrafted_model,
    plc_model,
    czt_model
)


import keras

FeatureLearnabilityTask.how_much = 3000 # 9k causes memory error


def small_network(self, source_shape, target_shape):
    source_shape = list(source_shape[1:])
    source_shape[0] = None
    mapping = self.get_mapping()
    mdl = keras.models.Sequential()
    mdl.add(keras.layers.Dense(512, activation='relu', input_shape=source_shape))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(target_shape[-1]))
    return mdl


def small_reg_network(self, source_shape, target_shape):
    source_shape = list(source_shape[1:])
    source_shape[0] = None
    mapping = self.get_mapping()
    mdl = keras.models.Sequential()
    mdl.add(keras.layers.Dense(512, activation='relu',
                               kernel_regularizer=keras.regularizers.L1L2(),
                               bias_regularizer=keras.regularizers.L1L2(),
                               input_shape=source_shape))
    mdl.add(keras.layers.Dense(512, activation='relu',
                               kernel_regularizer=keras.regularizers.L1L2(),
                               bias_regularizer=keras.regularizers.L1L2()))
    mdl.add(keras.layers.Dense(target_shape[-1]))
    return mdl
    
    
def large_network(self, source_shape, target_shape):
    source_shape = list(source_shape[1:])
    source_shape[0] = None
    mapping = self.get_mapping()
    mdl = keras.models.Sequential()
    mdl.add(keras.layers.Dense(512, activation='relu', input_shape=source_shape))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(512, activation='relu'))
    mdl.add(keras.layers.Dense(target_shape[-1]))
    return mdl
    
for mapper_cb in [small_network, large_network, small_reg_network]:

    class MFCCLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return mfcc_model()

        get_mapper_network = classmethod(mapper_cb)


    class PLCLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return plc_model()

        get_mapper_network = classmethod(mapper_cb)


    class CZTLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return czt_model()

        get_mapper_network = classmethod(mapper_cb)


    class HandcraftedLearnability(metaclass=FeatureLearnabilityTask):
        @classmethod
        def get_mapping(self):
            return handcrafted_model()

        get_mapper_network = classmethod(mapper_cb)

    for task in Task.all():
        print(task, mapper_cb.__name__)
        task.run("/tmp/cache")


Yet another class to be finished: TrainableCZT
Yet another class to be finished: TuningCurves
Yet another class to be finished: RoEx
Yet another class to be finished: CARFAC
Yet another class to be finished: GammaChirp
Yet another class to be finished: TimeRoex
Yet another class to be finished: TrainableWavelet
Yet another class to be finished: CARFAC
Yet another class to be finished: AntimonotonyLayer
Yet another class to be finished: SpectralCoefficients
Yet another class to be finished: Pointwise
Yet another class to be finished: AbstractWavelet
Yet another class to be finished: AbstractFilter
Yet another class to be finished: ExcitationTrace
Yet another class to be finished: WER
<class '__main__.CZTLearnability'> small_network


/venv/lib/python3.5/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
0it [00:00, ?it/s]

Getting dataset lengths


13810it [00:01, 9906.85it/s]
  0%|          | 0/3000 [00:00<?, ?it/s]fwks_lib/fwks/stage/stage_transforms.py:192: ComplexWarning: Casting complex values to real discards the imaginary part
  mapped[i] = spfft.dct(recording[i], n=self.num)
  0%|          | 1/3000 [00:00<07:10,  6.96it/s]

Getting clean recordings


...

13810it [00:01, 9580.45it/s]
  2%|▏         | 59/3000 [00:00<00:05, 584.18it/s]

Getting clean recordings


100%|██████████| 3000/3000 [00:07<00:00, 379.78it/s]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, None, 512)         262656    
_________________________________________________________________
dense_5 (Dense)              (None, None, 512)         262656    
_________________________________________________________________
dense_6 (Dense)              (None, None, 24)          12312     
Total params: 537,624
Trainable params: 537,624
Non-trainable params: 0
_________________________________________________________________
Train on 2387 samples, validate on 613 samples
Epoch 1/250


InternalError: Blas GEMM launch failed : a.shape=(42528, 512), b.shape=(512, 512), m=42528, n=512, k=512
	 [[node dense_4/MatMul (defined at /venv/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1070)  = MatMul[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/dense_4/MatMul_grad/MatMul_1"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_4/Reshape, dense_4/kernel/read)]]
	 [[{{node loss_1/mul/_209}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_523_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'dense_4/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  